<a href="https://colab.research.google.com/github/sugarforever/LangChain-Advanced/blob/main/autogen_langchain_uniswap_ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen + LangChain Use Case - Uniswap Protocol AI Agent

**`AutoGen`** is a versatile framework that facilitates the creation of LLM applications by employing multiple agents capable of interacting with one another to tackle tasks. These AutoGen agents can be tailored to specific needs, engage in conversations, and seamlessly integrate human participation. They are adaptable to different operation modes that encompass the utilization of LLMs, human inputs, and various tools.

**`LangChain`** is an open-source framework designed for software developers engaged in AI and ML. It enables them to seamlessly integrate LLM with external components, facilitating the creation of LLM-driven applications. The primary aim of LangChain is to establish connections between LLMs such as OpenAI's GPT-3.5 and GPT-4 and various external data sources, enabling the development and utilization of NLP applications.

Both of them are playing key roles in the LLM application development.

`AutoGen` doesn't support connecting to various external data sources natively. This is exactly where `LangChain` can come into play.

## Use Case - Uniswap Protocol

`Uniswap` is a decentralized exchange that allows users to trade Ethereum-based tokens.

In this tutorial, I will walk you through the steps to follow to build an AI agent that can complete tasks that involve Uniswap knowledge.

1. Build up a vector store with Uniswap V3 whitepaper.
2. Set up a conversational retrieval QA chain by LangChain, based on the vector store.
3. Define a function `answer_uniswap_question` by taking a parameter `question`, calling the QA chain to answer it.
4. Set up AutoGen user agent and assistant agent with function calling enabled.

  - In the function mapping, the function defined in step 3 is included.
  - The assistant agent is instructed by the following message to call the function `answer_uniswap_question` to answer Uniswap related questions.

    ```
    I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

    1. What is Uniswap?
    2. What are the main changes in Uniswap version 3?
    3. How to use Uniswap?

    Start the work now.
    ```

  - The function call is done by user agent.

### Environment Preparation

In [ ]:
%pip install pyautogen~=0.1.0 docker langchain openai tiktoken chromadb pypdf -q -U

In [ ]:
!wget -O uniswap_v3.pdf https://uniswap.org/whitepaper-v3.pdf

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)
#
# Sample content of OAI_CONFIG_LIST file below:
#
# [
#   {
#     "model": "gpt-4",
#     "api_key": "your openai api key"
#   }
# ]
#

In [ ]:
gpt4_api_key = config_list[0]["api_key"]

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = gpt4_api_key

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

### Steps

#### 1. Build up a vector store with Uniswap V3 whitepaper.

In [ ]:
loaders = [ PyPDFLoader('./uniswap_v3.pdf') ]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [ ]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings()
)
vectorstore.add_documents(docs)

#### 2. Set up a conversational retrieval QA chain by LangChain, based on the vector store.

In [ ]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [ ]:
result = qa(({"question": "What is uniswap?"}))

In [ ]:
result['answer']

#### 3. Define a function `answer_uniswap_question`

It takes a parameter `question`, calls the QA chain, and answer it by returning the answer from the chain response.

In [ ]:
def answer_uniswap_question(question):
  response = qa({"question": question})
  return response["answer"]

#### 4. Set up AutoGen user agent and assistant agent with function calling enabled.

In [ ]:
llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_uniswap_question",
            "description": "Answer any Uniswap related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to Uniswap protocol",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [ ]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_uniswap_question": answer_uniswap_question}
)

### It's time to start a task for the agents.

Now, let's user the user agent to ask the agents to write an introduction blog for `Uniswap` protocol v3.

In [ ]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.
"""
)